# DKW Controller Evaluation

This notebook evaluates the performance of a DKW Controller, comparing baseline and proposed methods. The evaluation focuses on:
- **Fusion vs Fission decisions**: The controller can choose to fuse or split operations
- **API efficiency**: Fusion requires 1 API call, fission requires 2 API calls
- **Error rates**: How often the controller makes incorrect decisions

The goal is to measure the improvement in API efficiency while maintaining acceptable error rates.

In [ ]:
import json
import numpy as np

## Sample Data

Instead of reading from external JSON files, we'll create sample data inline that represents the experimental results from both baseline and proposed methods.

In [ ]:
# Create sample experimental results
# This data represents the decisions made by baseline vs proposed methods

# Baseline method: always chooses fission, 8% error rate
baseline_results = []
for i in range(200):
    baseline_results.append({
        "decision": "fission",
        "error": i < 16  # First 16 examples have errors (8% of 200)
    })

# Proposed method: 65% fusion, 35% fission, 9% error rate  
proposed_results = []
for i in range(200):
    if i < 130:  # First 130 examples use fusion (65% of 200)
        decision = "fusion"
    else:  # Remaining 70 examples use fission (35% of 200)
        decision = "fission"
    
    proposed_results.append({
        "decision": decision,
        "error": i < 18  # First 18 examples have errors (9% of 200)
    })

# Combine into results structure
results = {
    "baseline": baseline_results,
    "proposed": proposed_results
}

print(f"Generated {len(baseline_results)} baseline results and {len(proposed_results)} proposed results")
print(f"Baseline decisions: {sum(1 for r in baseline_results if r['decision'] == 'fusion')} fusion, {sum(1 for r in baseline_results if r['decision'] == 'fission')} fission")
print(f"Proposed decisions: {sum(1 for r in proposed_results if r['decision'] == 'fusion')} fusion, {sum(1 for r in proposed_results if r['decision'] == 'fission')} fission")

## Evaluation Metrics Function

The `compute_metrics` function calculates several key performance indicators:

- **Fusion/Fission rates**: Proportion of decisions for each strategy
- **Error rate**: Percentage of incorrect decisions
- **API calls**: Total API usage (fusion = 1 call, fission = 2 calls)
- **Efficiency improvements**: Comparison between methods

In [ ]:
def compute_metrics(results: dict) -> dict:
    """Compute evaluation metrics."""
    metrics = {}

    for method in ["baseline", "proposed"]:
        preds = results[method]

        # Count decisions
        fusion_count = sum(1 for p in preds if p["decision"] == "fusion")
        fission_count = sum(1 for p in preds if p["decision"] == "fission")

        # Compute error rate
        errors = sum(1 for p in preds if p["error"])
        error_rate = errors / len(preds)

        # API calls (fusion=1, fission=2)
        api_calls = fusion_count + 2 * fission_count

        metrics[method] = {
            "fusion_rate": fusion_count / len(preds),
            "fission_rate": fission_count / len(preds),
            "error_rate": error_rate,
            "api_calls": api_calls,
            "avg_calls_per_example": api_calls / len(preds),
        }

    # Compute improvement
    baseline_calls = metrics["baseline"]["avg_calls_per_example"]
    proposed_calls = metrics["proposed"]["avg_calls_per_example"]
    metrics["improvement"] = {
        "api_reduction_pct": (baseline_calls - proposed_calls) / baseline_calls * 100,
        "error_rate_diff": metrics["proposed"]["error_rate"] - metrics["baseline"]["error_rate"],
    }

    return metrics

## Run Evaluation

Now let's compute the metrics and display the results:

In [ ]:
# Compute metrics
metrics = compute_metrics(results)

# Display results in a formatted way
print("="*50)
print("DKW CONTROLLER EVALUATION RESULTS")
print("="*50)

for method in ["baseline", "proposed"]:
    print(f"\n{method.upper()} METHOD:")
    m = metrics[method]
    print(f"  Fusion rate:     {m['fusion_rate']:.1%}")
    print(f"  Fission rate:    {m['fission_rate']:.1%}") 
    print(f"  Error rate:      {m['error_rate']:.1%}")
    print(f"  Total API calls: {m['api_calls']}")
    print(f"  Avg calls/example: {m['avg_calls_per_example']:.2f}")

print(f"\nIMPROVEMENT:")
print(f"  API reduction: {metrics['improvement']['api_reduction_pct']:.1f}%")
print(f"  Error rate change: {metrics['improvement']['error_rate_diff']:+.1%}")

# Also save as JSON (inline output)
print(f"\nFull metrics as JSON:")
print(json.dumps(metrics, indent=2))

## Experiment with Different Scenarios

You can modify the parameters below to see how different controller behaviors would affect the metrics:

In [ ]:
# Experiment: What if we had a different proposed method?
# Modify these parameters to see different scenarios

def create_experimental_results(n_examples=200, fusion_rate=0.8, error_rate=0.05):
    """Create experimental results with custom parameters."""
    results = []
    n_fusion = int(n_examples * fusion_rate)
    n_errors = int(n_examples * error_rate)
    
    for i in range(n_examples):
        decision = "fusion" if i < n_fusion else "fission"
        error = i < n_errors
        results.append({"decision": decision, "error": error})
    
    return results

# Try different scenarios
experimental_scenarios = {
    "baseline": baseline_results,  # Keep baseline the same
    "high_fusion_low_error": create_experimental_results(fusion_rate=0.9, error_rate=0.03),
    "medium_fusion": create_experimental_results(fusion_rate=0.5, error_rate=0.06),
    "original_proposed": proposed_results
}

print("Comparing different scenarios:\n")
for scenario_name, scenario_results in experimental_scenarios.items():
    if scenario_name == "baseline":
        continue
    
    scenario_data = {"baseline": baseline_results, "proposed": scenario_results}
    scenario_metrics = compute_metrics(scenario_data)
    
    print(f"{scenario_name.upper()}:")
    print(f"  API reduction: {scenario_metrics['improvement']['api_reduction_pct']:.1f}%")
    print(f"  Error rate change: {scenario_metrics['improvement']['error_rate_diff']:+.1%}")
    print(f"  Fusion rate: {scenario_metrics['proposed']['fusion_rate']:.1%}")
    print()